# PromptPilot: prompt store and execution

Here we develop a DDE (Data Driven Execution object) around prompt templates. 
A prompt template is some text where some parts are placeholders where we will inject 
text to make the final prompt. Example: "suggest names for {this: a random subject}".

Users can 
* do CRUD on prompt templates,
* execute these templates, which means
    * inject placeholder "embodiments"
    * send "embodied" prompts to an LLM to get a response

In code it looks like this:

```python
from oa import prompt_function

template = "suggest names for {this: a random subject}"

# Embody template only
f = prompt_function(template, prompt_func=None)  # prompt_func=None means "no LLM"
f(this='A superhero that knows JS')
# Returns: 'suggest names for A superhero that knows JS'

# Embody and execute
f(this='A superhero that knows JS')
# Returns:
#   1. Code Crusader
#   2. JavaScript Jedi
#   3. The Script Slinger
#   4. Byte Buster
#   5. The JavaScript Avenger
```


## Example using prompt function

In [5]:
from oa import prompt_function

template = "suggest names for {this: a random subject}"


In [2]:
f = prompt_function(template, prompt_func=None)  # prompt_func=None means "no LLM"
f(this='A superhero that knows JS')

'suggest names for A superhero that knows JS'

In [4]:
f = prompt_function(template)  # 
print(f(this='A superhero that knows JS'))  # this will use an llm (by default an openai model)

1. Code Crusader
2. JavaScript Jedi
3. The Script Slinger
4. Byte Buster
5. The JavaScript Avenger
6. The Web Warrior
7. Syntax Savior
8. The Programming Prodigy
9. The Tech Titan
10. The Code Commander


## Trying the dde object directly in python

Here we show the `prompt_template_dde` object that will act as our controller for the prompt templates app.

Creating an app that enables users to perform CRUD operations on "prompt templates" for use with Large Language Models (LLMs) involves blending concepts of template management, user input customization, and AI execution. The app's name should reflect its functionality of crafting, managing, and executing customized prompts. Here are several suggestive names that encapsulate the app’s core features and appeal to its target audience:

PromptCraft: Emphasizes the creation and customization of prompt templates (Craft) for execution with LLMs.
PromptForge: Suggests the shaping and forging of customizable prompt templates for use in AI-driven executions.
PromptPilot: Indicates guiding or piloting the creation and execution of prompt templates, emphasizing control and direction in AI interactions.
TemplateTuner: Indicates fine-tuning or adjusting templates for optimal interactions with LLMs, emphasizing customization and precision.
QueryQuiver: Uses alliteration to suggest a collection or "quiver" of queries or prompts ready to be deployed with LLMs.


In [6]:
from raglab.web_services.prompt_templates_dde import prompt_template_dde
from raglab.web_services.prompt_templates_dde import *

print(f"The data is stored here:\n\t{prompt_template_dde.store.rootdir}")

The data is stored here:
	/Users/thorwhalen/.config/raglab/configs/data/spaces/prompt_tests/stores/prompt_templates/


In [31]:
prompt_template_dde.write('les_Ms', 'Tell me a joke about {subject}')

In [32]:
prompt_template_dde.list()

['user_story_to_code_for_dag',
 'make_synopsis',
 'description_to_software_specs',
 'define_jargon',
 'simple_tests_for_code',
 'les_Ms',
 'suggest_names',
 'specs_to_code_for_dag']

In [20]:
prompt_template = prompt_template_dde.read('suggest_names')
print(prompt_template)

Suggest {n:30} names 
between {min_length:1} and {max_length:15} characters long for {thing}.
Only output the names, one per line with no words before or after it, 
since I will be parsing the output.


In [33]:
result = prompt_template_dde.execute_prompt_from_key(
    'les_Ms', {'subject': 'Frontend development versus backend development'}
)
print(result)


Why did the frontend developer break up with the backend developer? Because they couldn't handle their constant requests for more space!


In [22]:
result = prompt_template_dde.execute_prompt(prompt_template, {'thing': 'A baby born on Feb 22'})
print(result)

Luna
Jade
Finn
Max
Eva
Grace
Leo
Zoe
Kai
Ivy
Eli
Sky
Nora
Cade
Mia
Jack
Ava
Owen
Lana
Faye
Cole
Zara
Tate
Liv
Nate
Maya
Rey
Joss
Zia
Xan


## Getting a function from a prompt template

In [46]:
func = prompt_template_dde.prompt_func("""Suggest {n:30} names 
between {min_length:1} and {max_length:15} characters long for {thing}.
Only output the names, one per line with no words before or after it, 
since I will be parsing the output.""")

func.__doc__ = """Suggest names"""

assert callable(func)  # func is a function!!

help(func)

Help on function ask_oa in module oa.tools:

ask_oa(thing, *, n='30', min_length='1', max_length='15')
    Suggest names



## Getting a rjsf json from a function

In [47]:
from ju.rjsf import func_to_form_spec
import json

from_spec = func_to_form_spec(func)
print(json.dumps(from_spec, indent=2))

{
  "rjsf": {
    "schema": {
      "title": "ask_oa",
      "type": "object",
      "properties": {
        "thing": {
          "type": "string"
        },
        "n": {
          "type": "string",
          "default": "30"
        },
        "min_length": {
          "type": "string",
          "default": "1"
        },
        "max_length": {
          "type": "string",
          "default": "15"
        }
      },
      "required": [
        "thing"
      ],
      "description": "Suggest names"
    },
    "uiSchema": {
      "ui:submitButtonOptions": {
        "submitText": "Run"
      },
      "thing": {
        "ui:autofocus": true
      }
    },
    "liveValidate": false,
    "disabled": false,
    "readonly": false,
    "omitExtraData": false,
    "liveOmit": false,
    "noValidate": false,
    "noHtml5Validate": false,
    "focusOnFirstError": false,
    "showErrorList": "top"
  }
}


## Calling the dde from a webservice

Well, first you need to launch the webservice by doing:

```
python PATH_TO/raglab/raglab/web_services/prompt_templates_dde.py
```

Note what the rooturl is, because the default in the function below is 'http://localhost:3030'

In [49]:
def call_obj_dispatch_web_service(
        attr='list', 
        params=None, *, 
        suburl = '/prompt_templates', 
        rooturl = 'http://localhost:3030'
    ):
    import requests

    url = f'{rooturl}{suburl}'
    params = params or {}

    add_args = dict(_attr_name=attr, **params)

    return requests.post(url, json=add_args).json()


In [50]:
call_obj_dispatch_web_service('list')

['user_story_to_code_for_dag',
 'make_synopsis',
 'description_to_software_specs',
 'define_jargon',
 'simple_tests_for_code',
 'les_Ms',
 'suggest_names',
 'specs_to_code_for_dag']

In [29]:
r = call_obj_dispatch_web_service(
    'execute_prompt_from_key', 
    {'key': 'suggest_names', 'params': {'thing': 'A baby born on Feb 22'}}
)
r

'Mia\nEli\nAva\nKai\nZoe\nLuke\nEva\nMax\nJax\nLia\nLeo\nMay\nRex\nRose\nIan\nFaye\nNia\nSky\nEve\nAsh\nJay\nLux\nBrynn\nKip\nNyx\nSasha\nZara\nTy\nLiv\nJet'

In [ ]:
from lkj import add_attr

# Making malls

## Local stores

In [1]:
from dol import JsonFiles, wrap_kvs, mk_dirs_if_missing, temp_dir, KeyPath, KeyTemplate, filt_iter, Files
import os

pjoin = os.path.join

In [2]:
# temporary directory

rootdir = temp_dir('ragdag/rjsf_fiddle')

print(rootdir)

/var/folders/mc/c070wfh51kxd9lft8dl74q1r0000gn/T/ragdag/rjsf_fiddle


In [33]:
from raglab.web_services.store_access import user_info_store, app

list(user_info_store)

['bob', 'alice']

In [5]:
# Test the server in a separate process
import requests

url = 'http://localhost:3030/add'
add_args = {'a': 20, 'b': 22}
requests.post(url, json=add_args).json()
# should return 42


42

In [ ]:
curl -X 'POST' \
  'http://localhost:8080/backend_store' \
  -H 'accept: application/json' \
  -H 'Content-Type: application/json' \
  -d '{
  "_attr_name": "__iter__"
}'

NameError: name 'app' is not defined

In [30]:
rooturl = 'http://localhost:8080'
url = f'{rooturl}/backend_store'

add_args = {
  "_attr_name": '__iter__',
}
assert requests.post(url, json=add_args).json() == ['some_key', 'some_other_key']

add_args = {
  "_attr_name": '__getitem__',
  "key": 'some_key'
}
assert requests.post(url, json=add_args).json() == 'some_value'

# Demoing pieces

## Make a temp directory

In [69]:
# temporary directory
from dol import temp_dir

rootdir = temp_dir('ragdag/rjsf_fiddle')

print(rootdir)

/var/folders/mc/c070wfh51kxd9lft8dl74q1r0000gn/T/ragdag/rjsf_fiddle


In [71]:
from raglab.util import local_stores_dir

local_stores_dir

'/Users/thorwhalen/.config/raglab/configs/local_stores'

## user_store_factories

In [1]:
from raglab.stores.stores_util import mk_local_user_store

In [26]:
from raglab.stores.simple_stores import user_store_factories, mk_user_mall, mk_user_mall_namespace
from i2 import Sig

mall = mk_user_mall_namespace('bob')
list(mall)
mall.py_objects['test'] = [1, b'bytes', 'text', (1, 2), mk_user_mall_namespace]
'test' in list(mall.py_objects)
assert mall.py_objects['test'] == [1, b'bytes', 'text', (1, 2), mk_user_mall_namespace]

TypeError: mk_user_mall() missing 1 required positional argument: 'user_name'

In [22]:
from raglab.web_services.store_access import *

store = {'apple': 1, 'banana': 2}

StoreAccess(store)




NameError: name 'rootdir' is not defined

In [25]:
mall = get_user_mall('bob')
mall

TypeError: 'StoreAccess' object is not iterable

In [21]:
from raglab.stores.stores_util import mk_space_info_store
from raglab.web_services.store_access import get_user_mall

# mall = get_user_mall()
# list(mall)


In [13]:
from raglab.web_services.store_access import user_info_store

list(user_info_store)

# user_info_store['bob'] = {'user': 'bob'}
user_info_store.rootdir

u_rootdir = '/Users/thorwhalen/.config/raglab/configs/local_stores/u/'
from dol import JsonFiles, mk_dirs_if_missing
s = mk_dirs_if_missing(JsonFiles(u_rootdir))
s['bob/info.json'] = {'user': 'bob'}
s['alice/info.json'] = {'user': 'alice'}
list(s)

['bob/info.json', 'alice/info.json']

## local_user_store

In [2]:
from dol import temp_dir

rootdir = temp_dir('ragdag/rjsf_fiddle')


In [4]:
rootdir

'/var/folders/mc/c070wfh51kxd9lft8dl74q1r0000gn/T/ragdag/rjsf_fiddle'

In [9]:
from raglab.stores.stores_util import *

f = mk_text_store(rootdir)
s = f('ana')
s['test'] = 'this is text'
list(s)

['test']

In [29]:
from raglab.stores.stores_util import *
from dol import temp_dir


def test_mk_local_user_store(verbose=False):

    rootdir = temp_dir('ragdag/rjsf_fiddle')
    if verbose:
        print(f"{rootdir=}")

    s = mk_local_user_store(rootdir, user_name='bob', store_kind='spec_1')
    s['some_key'] = b'some_value'
    s['some_other_key'] = b'some_other_value'
    if verbose:
        print(f"Go check out this folder: file://{s.rootdir}")
    assert 'some_key' in s
    assert s['some_key'] == b'some_value'
    assert 'some_key' in os.listdir(s.rootdir)

def test_mk_specialized_local_user_store(verbose=False):

    rootdir = temp_dir('ragdag/rjsf_fiddle')
    if verbose:
        print(f"{rootdir=}")

    s = mk_json_store(rootdir, user_name='alice', store_kind='jsons')
    s['some_key'] = {'a': 'b'}
    assert s['some_key'] == {'a': 'b'}

    # see that the file is there, with a json extension
    assert 'some_key.json' in os.listdir(s.rootdir)
    # see that the file contains the expected content, as json string
    from pathlib import Path
    assert Path(os.path.join(s.rootdir, 'some_key.json')).read_text() == '{"a": "b"}'

    # showing how, when user_name is not provided, mk_text_store returns a store factory
    # with the rootdir fixed
    mk_text_store_for_user = mk_text_store(rootdir)
    s = mk_text_store_for_user('alice', store_kind='texts')
    s['some_key'] = 'some_value'
    assert s['some_key'] == 'some_value'

    # we can fix any attribute of the store factory
    mk_pickle_store_for_user = mk_pickle_store(rootdir, store_kind='pickles')
    s = mk_pickle_store_for_user(user_name='alice')
    ecclectic_data = [1, 2.3, 'four', map]
    s['some_key'] = ecclectic_data
    assert s['some_key'] == ecclectic_data

test_mk_local_user_store(verbose=True)
test_mk_specialized_local_user_store()


rootdir='/var/folders/mc/c070wfh51kxd9lft8dl74q1r0000gn/T/ragdag/rjsf_fiddle'
Go check out this folder: file:///var/folders/mc/c070wfh51kxd9lft8dl74q1r0000gn/T/ragdag/rjsf_fiddle/u/bob/stores/spec_1/


## dol.KeyTemplate

In [ ]:
## Making a key template

from dol import KeyTemplate
import os

pjoin = os.path.join

user_template = pjoin('u', '{user_name}')
stores_template = pjoin('stores', '{store_name}')

kt = KeyTemplate(template=pjoin(user_template, stores_template))
key_wrap = kt.key_codec(decoded='tuple', encoded='str')

back = {
    'u/bob/stores/spec_1': {'an': 'apple'},
    'u/bob/stores/spec_2': {'an': 'orange'},
    'u/alice/stores/spec_1': {'i_like': 'to move it'},
    'some/thing/else': 'contents'
}

front = key_wrap(kt.filt_iter(back))
assert sorted(front) == [('alice', 'spec_1'), ('bob', 'spec_1'), ('bob', 'spec_2')]
front['bob', 'spec_1']